## Imports

In [1]:
# Basic imports
import pandas as pd
import numpy as np

# Warnings
import warnings 
warnings.simplefilter("ignore")

In [2]:
# Plot
from IPython import display
import seaborn as sns

import matplotlib
import matplotlib.pylab as plt
from jupyterthemes import jtplot

jtplot.style('gruvboxd')
matplotlib.use('nbagg')

# Data Reading

In [3]:
from catboost.datasets import titanic

# Data Reading
df_train, df_test = titanic()
df_train.set_index('PassengerId', inplace=True)
df_test.set_index('PassengerId', inplace=True)

# Split X_train, y_train
target = 'Survived'
features = df_test.columns

y_train = df_train[target]
df_train = df_train[features]

df_train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Data Preparation

In [4]:
from robusta.preprocessing.category import *
from robusta.preprocessing.numeric import *
from robusta.preprocessing import *
from robusta.compose import *

nums = ['Age', 'Fare', 'SibSp', 'Parch']
cats = ['Pclass', 'Sex', 'Embarked']

data_prep = FeatureUnion([
        ("numeric", make_pipeline(
            ColumnSelector(nums),
            Imputer(strategy="median"),
            GaussRank(),
            #ColumnRenamer(prefix='gr_'),
        )),
        ("category", make_pipeline(
            ColumnSelector(cats),
            LabelEncoder(dtype='category'),
            #ColumnRenamer(prefix='le_'),
        )),
])

X_train = data_prep.fit_transform(df_train)
X_test = data_prep.transform(df_test)

X_train.head()

,Age,Fare,SibSp,Parch,Pclass,Sex,Embarked
PassengerId,,,,,,,
1,-0.488867,-0.968076,0.600843,-0.214091,2,1,2
2,0.584412,0.853865,0.600843,-0.214091,0,0,0
3,-0.275357,-0.454031,-0.290348,-0.214091,2,0,2
4,0.470091,0.703973,0.600843,-0.214091,0,0,2
5,0.470091,-0.379298,-0.290348,-0.214091,2,1,2


# Fold Preparation

In [5]:
%%time
from lightgbm import LGBMClassifier

estimator = LGBMClassifier()
estimator.fit(X_train, y_train)

CPU times: user 417 ms, sys: 10.3 ms, total: 428 ms
Wall time: 75.3 ms


/Users/pechyonkinb/anaconda/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [9]:
from sklearn.model_selection import RepeatedKFold

#te_cv = RepeatedKFold(5, 3)
te_cv = 4

fold_prep = FeatureUnion([
    ('category', make_pipeline(
        TypeSelector(['category', 'object']), 
        TargetEncoderCV(cv=te_cv, smoothing=1))),
    ('numeric', TypeSelector(np.number))
])
#fold_prep = ColumnTransformer([
#    ('cats', TargetEncoderCV(cv=te_cv, smoothing=1), cats),
#    ('nums', Identity(), nums)
#])
    
F_train = fold_prep.fit_transform(X_train, y_train)

F_train.sample(5, random_state=555)

,Embarked,Pclass,Sex,Age,Fare,SibSp,Parch
PassengerId,,,,,,,
876,0.539062,0.250000,0.739669,-0.944113,-1.132927,-0.290348,-0.214091
595,0.336082,0.442857,0.177986,0.560940,0.277085,0.600843,-0.214091
133,0.334711,0.217514,0.741379,0.888832,0.016949,0.600843,-0.214091
233,0.341615,0.500000,0.201357,1.332614,-0.027920,-0.290348,-0.214091
506,0.532787,0.629870,0.177986,-0.757656,1.132927,0.600843,-0.214091


In [10]:
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import *

model = make_pipeline(fold_prep, estimator)

cv = 5
#scoring = 'neg_log_loss'
scoring = 'accuracy'

scores = cross_val_score(model, X_train, y_train, cv=cv, scoring=scoring)

print('{:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores)))

0.8272 ± 0.0220


In [11]:
model.get_params()

{'memory': None,
 'steps': [('featureunion',
   FeatureUnion(transformers=[('category', Pipeline(memory=None,
        steps=[('typeselector', TypeSelector(dtype=['category', 'object'])), ('encodercv', EncoderCV(cv=4,
        encoder=TargetEncoder(cols=None, drop_invariant=False, handle_missing='value',
          handle_unknown='value', min_samples_leaf=1, return_df=True,
          smoothing=1.0, verbose=0),
        n_jobs=-1, verbose=0))])), ('numeric', TypeSelector(dtype=<class 'numpy.number'>))])),
  ('lgbmclassifier',
   LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
           importance_type='split', learning_rate=0.1, max_depth=-1,
           min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
           n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
           random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
           subsample=1.0, subsample_for_bin=200000, subsample_freq=0))],
 'featureunion': FeatureUnio